In [1]:
import open3d as o3d
import numpy as np
import math

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd = o3d.io.read_point_cloud("/home/aichi2204/ros2_ws/test/pcl-test/data/map_all_removed/best2/39.pcd")

In [3]:
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
          center=pcd.get_center())

PointCloud with 2502547 points.

In [19]:
octree = o3d.geometry.Octree(max_depth=5)
octree.convert_from_point_cloud(pcd, size_expand=0.01)

In [20]:
# pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(2000, 3)))
# o3d.visualization.draw_geometries([pcd])
# o3d.visualization.draw_geometries([octree])

In [32]:
node = octree.locate_leaf_node(point)
node

(OctreePointColorLeafNode with color [0.105882, 0.105882, 0.0784314] containing 2 points.,
 OctreeNodeInfo with origin [-50.3946, -8.67925, -1.32178], size 0.0315625, depth 5, child_index 3)

In [46]:
# 指定した点のボクセルを取得する関数
def find_voxel_for_point(octree, point):
    result = octree.locate_leaf_node(point)
    if result is not None:
        node, _ = result
        return node
    else:
        return None

# ノードの中心座標を取得する関数
def get_voxel_center(node, min_bound, voxel_size):
    center = min_bound + (np.asarray(node.origin) + voxel_size / 2.0)
    return center

# 近傍のボクセルを取得する関数
def find_neighbor_voxels(octree, node, min_bound, voxel_size):
    center = get_voxel_center(node, min_bound, voxel_size)
    neighbors = []
    directions = [
        np.array([1, 0, 0]), np.array([-1, 0, 0]),
        np.array([0, 1, 0]), np.array([0, -1, 0]),
        np.array([0, 0, 1]), np.array([0, 0, -1])
    ]
    
    for direction in directions:
        neighbor_center = center + direction * voxel_size
        result = octree.locate_leaf_node(neighbor_center)
        if result is not None:
            neighbor_node, _ = result
            neighbors.append(neighbor_node)
    
    return neighbors


point = pcd.points[0]
voxel = find_voxel_for_point(octree, point)

if voxel is not None:
    min_bound = pcd.get_min_bound()
    max_bound = pcd.get_max_bound()
    voxel_size = (max_bound - min_bound) / (2 ** octree.max_depth)
    
    neighbors = find_neighbor_voxels(octree, voxel, min_bound, voxel_size)
    print(f"Neighbors of the voxel containing {point}:")
    for neighbor in neighbors:
        print(get_voxel_center(neighbor, min_bound, voxel_size))
else:
    print(f"Point {point} is not in any voxel")

AttributeError: 'open3d.cuda.pybind.geometry.OctreePointColorLeafNo' object has no attribute 'origin'

In [61]:
point = pcd.points[0]
voxel = find_voxel_for_point(octree, point)
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()
voxel_size = (max_bound - min_bound) / (2 ** octree.max_depth)

center = get_voxel_center(voxel, min_bound, voxel_size)

In [118]:
# voxel 内の点をすべて抽出
point = pcd.points[100]
node, node_info = octree.locate_leaf_node(point)
node_points = np.asarray(pcd.points)[node.indices]

neighbors = []
directions = [
    np.array([1, 0, 0]), np.array([-1, 0, 0]),
    np.array([0, 1, 0]), np.array([0, -1, 0]),
    np.array([0, 0, 1]), np.array([0, 0, -1])
]

for direction in directions:
    neighbor_center = node_info.origin + direction * node_info.size
    result = octree.locate_leaf_node(neighbor_center)
    if result is not None:
        neighbor_node, _ = result
        neighbors.append(neighbor_node)

print(neighbors)

[OctreePointColorLeafNode with color [0.211765, 0.219608, 0.2] containing 213 points., None, OctreePointColorLeafNode with color [0.133333, 0.129412, 0.113725] containing 302 points., OctreePointColorLeafNode with color [0.227451, 0.223529, 0.196078] containing 63 points., None, None]


In [119]:
octree

Octree with origin: [-50.4262, -8.83706, -1.76366], size: 1.01, max_depth: 5